Jina AI: (repo can be found at https://github.com/jina-ai/serve) \
Use r.jina.ai to read any URL and convert into LLM-friendly text - this includes:
* Title & URL specified
* Subheadings specified by ** Subheading ** with all text expanded and written below
* Hyperlinks are expanded into full URL (useful for document cross-referencing)\
  
Note that headings are *not* removed

All cells run on Python. Standard Python URL parsing uses requests.\
Using cache helps (a) reduce the number of API keys being used in the cases where tokens may be limited and (b) reduce run time in cases where the same page is often fetched (would be the case for the files containing elementary environmental legislation.)

In [ ]:
import requests
from functools import cache

In [ ]:
# Below is a list of all the URLs provided in the task description. The one uncommented does indeed refer to environmental legislation.

# Ministry of Energy Act, 2011, S.O. 2011, c. 9, Sched. 25 
# url = 'https://www.ontario.ca/laws-beta/statute/11m09a?highlight=true&lang=en&option=%7B%22selection%22%3A%5B%22current%22%5D%2C%22result%22%3A%5B%22statute%22%2C%22regulation%22%5D%7D&paging=%7B%22page%22%3A1%2C%22pageSize%22%3A50%2C%22sort%22%3A%22Relevance%22%7D&searchMode=search&searchWithin=%5B%22title%22%2C%22body%22%5D&searchWithinResult=false&selection=consolidated+law&text=Energy&trigger=0&useExact=false&withinResultSearch='

#Environmental Protection Act, R.S.O. 1990, c. E.19
# url = 'https://www.ontario.ca/laws-beta/statute/10e08?highlight=false&lang=en&option=%7B%22selection%22%3A%5B%22current%22%5D%2C%22result%22%3A%5B%22statute%22%2C%22regulation%22%5D%7D&paging=%7B%22page%22%3A1%2C%22pageSize%22%3A50%2C%22sort%22%3A%22AZ%22%7D&searchMode=browse&searchWithin=%5B%22title%22%2C%22body%22%5D&searchWithinResult=false&selection=consolidated+law&text=&trigger=1&useExact=false&withinResultSearch='

# Keeping Polystyrene Out of Ontario's Lakes and Rivers Act, 2021, S.O. 2021, c. 16 
# url = 'https://www.ontario.ca/laws-beta/statute/21k16?highlight=false&lang=en&option=%7B%22selection%22%3A%5B%22current%22%5D%2C%22result%22%3A%5B%22statute%22%2C%22regulation%22%5D%7D&paging=%7B%22page%22%3A1%2C%22pageSize%22%3A50%2C%22sort%22%3A%22AZ%22%7D&searchMode=browse&searchWithin=%5B%22title%22%2C%22body%22%5D&searchWithinResult=false&selection=consolidated+law&text=&trigger=1&useExact=false&withinResultSearch='

# Ministry of Agriculture, Food and Rural Affairs Act, R.S.O. 1990, c. M.16 
# url = 'https://www.ontario.ca/laws-beta/statute/90m16?highlight=false&lang=en&option=%7B%22selection%22%3A%5B%22current%22%5D%2C%22result%22%3A%5B%22statute%22%5D%7D&paging=%7B%22page%22%3A1%2C%22pageSize%22%3A50%2C%22sort%22%3A%22AZ%22%7D&searchMode=browse&searchWithin=%5B%22title%22%2C%22body%22%5D&searchWithinResult=false&selection=consolidated+law&text=&trigger=0&useExact=false&withinResultSearch='

# Electricity Act, 1998, S.O. 1998, c. 15, Sched. A  
# url = 'https://www.ontario.ca/laws-beta/statute/98e15?highlight=false&lang=en&option=%7B%22selection%22%3A%5B%22current%22%5D%2C%22result%22%3A%5B%22statute%22%2C%22regulation%22%5D%7D&paging=%7B%22page%22%3A1%2C%22pageSize%22%3A50%2C%22sort%22%3A%22AZ%22%7D&searchMode=browse&searchWithin=%5B%22title%22%2C%22body%22%5D&searchWithinResult=false&selection=consolidated+law&text=&trigger=1&useExact=false&withinResultSearch=#act-verion'

# Planning Act, R.S.O. 1990, c. P.13
# Count = 33
url = 'https://www.ontario.ca/laws-beta/statute/90p13'

# # Ministry of the Environment Act, R.S.O. 1990, c. M.24  
# url = 'https://www.ontario.ca/laws-beta/statute/90m24?highlight=true&lang=en&option=%7B%22selection%22%3A%5B%22current%22%5D%2C%22result%22%3A%5B%22statute%22%5D%7D&paging=%7B%22page%22%3A1%2C%22pageSize%22%3A50%2C%22sort%22%3A%22Relevance%22%7D&searchMode=search&searchWithin=%5B%22title%22%2C%22body%22%5D&searchWithinResult=false&selection=consolidated+law&text=environment&trigger=1&useExact=false&withinResultSearch='

# #Narcotics Safety and Awareness Act, 2010, S.O. 2010, c. 22
# url = 'https://www.ontario.ca/laws-beta/statute/10n22?highlight=false&lang=en&option=%7B%22selection%22%3A%5B%22current%22%5D%2C%22result%22%3A%5B%22statute%22%2C%22regulation%22%5D%7D&paging=%7B%22page%22%3A1%2C%22pageSize%22%3A50%2C%22sort%22%3A%22AZ%22%7D&searchMode=browse&searchWithin=%5B%22title%22%2C%22body%22%5D&searchWithinResult=false&selection=consolidated+law&text=&trigger=1&useExact=false&withinResultSearch='

key_words = {"energy", "environment"}

The below page request runs on Jina AI. Jina limits free API keys to 1,000,000 tokens, meaning that a few calls of the below exhaust capacity. It is a very reliable page parser and returns legible and necessary data, but is restricted by this property.

In [ ]:
!pip install jina

In [ ]:
# Cell execution time for single article - 22 seconds counting filtering - reduced to ~2 with cacheing


#api_key = 'jina_8d00676eed0f40f7a3cf174ede7a3efb0la7fUvEW2SSm4Cuk5Eao8nZ5Dtt'
# api_key = 'jina_2c94c91a6898439f9ff9b0f4434843eaWYxj-lcdXzqblO8k3VhAIoLCUnWh' # new key - use wisely 

@cache
def fetch(url):
    jina_url = 'https://r.jina.ai/' + url
    # headers = {
    #     'Authorization': 'Bearer ' + api_key     # Specific API key gives higher rate limit
    # }
    # response = requests.get(jina_url, headers=headers)
    response = requests.get(jina_url, headers=None)
    return response.text

In [ ]:
print(fetch(url))

In [ ]:
import re

def filter(url):
    readable = fetch(url)
    # count = sum(readable.count(word) for word in key_words)
    
    c = re.compile(r'\b(' + '|'.join(key_words) + r')\b', re.IGNORECASE)
    for line in readable.splitlines():
        if c.search(line):
            print(line)
    # if count > 5:
    #     print(count)
    #     print(readable)

In [ ]:
filter(url)

Alternative method uses standard Python Scrapy library. The problem with this library is that it is markdown-specific, so will return JSON only for the specified attributes. This is not portable across the range of legal files.

In [ ]:
!pip install scrapy

In [ ]:
@cache
def fetch(url):
    response = requests.get(url)
    return response.text

print(fetch(url))    

Llama, similar to Jina AI, is an API-based scraper that limits pages to 1000 per day (it is targeted to PDF scraping rather than web-based) and temporarily stores files for download before converting to readable format. \
Really does not work great.

In [ ]:
#Using LlamaParse
!pip install llama-index-core llama-index-readers-web html2text
api_key = 'llx-QsLfJvb2bc4M0bspacSTeDNjt0lBeFf6uAevXEYm30OKrflz'

In [ ]:
# For AI-specific web scraping: https://docs.llamaindex.ai/en/stable/examples/data_connectors/WebPageDemo/

from llama_index.readers.remote import RemoteReader
from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, HTML, display
import os

# loader = RemoteReader()
# documents = loader.load_data(
#     url=url
# )

documents = SimpleWebPageReader(html_to_text=True).load_data(
    [url]
)
# documents[0]
# index = SummaryIndex.from_documents(documents)
display(Markdown(f"{documents}"))

# print(documents)

Spider is the recommended (?) mode of web-crawling but also uses its own API and those keys cost money.

**BeautifulSoup** is a well-known manual web-scraping page. 
It has to use **Selenium** to simulate a JavaScript environment.

In [ ]:
!pip install beautifulsoup4
!pip install selenium

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from urllib.request import urlopen

service = Service()
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)
selector = '#law-content > div > div.laws-document__act-content'
WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, selector))
)

html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

print(soup.prettify())

driver.quit()
#-----------------------------------------------------

# driver = webdriver.Chrome()       #Specify exe path?
# driver.get(url)

# driver.implicitly_wait(20)

# html = driver.page_source

# # Parse with Beautiful Soup
# soup = BeautifulSoup(html, 'html.parser')

# # Now you can work with `soup`
# print(soup.prettify())

# # Close the browser
# driver.quit()

#---------------------------------------

# page = urlopen(url)
# html = page.read().decode("utf-8")
# soup = BeautifulSoup(html, "html.parser")

# print(soup.get_text())